### Create Cube: Hierarchies, levels and measures

In [1]:
# Connect to Olympics Database
import os
import psycopg2
from psycopg2 import OperationalError
from dotenv import load_dotenv
import psycopg2
from psycopg2 import OperationalError
 
# Load environment variables
load_dotenv()

def create_connection():
    connection = None
    try:
        connection = psycopg2.connect(
            database=os.getenv('DB_NAME'),
            user=os.getenv('DB_USER'),
            password=os.getenv('DB_PASSWORD'),
            host=os.getenv('DB_HOST'),
            port=os.getenv('DB_PORT'),
        )
        print("Connection to PostgreSQL DB successful")
    except OperationalError as e:
        print(f"The error '{e}' occurred")
    return connection

# Create the connection
conn = create_connection()
 

Connection to PostgreSQL DB successful


In [2]:
import pandas as pd
import atoti as tt
session = tt.Session()
#jdbc_url = f"jdbc:postgresql://{db_host}:{db_port}/{db_name}?user={db_user}&password={db_password}"


C:\Users\61470\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\61470\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


Welcome to Atoti 0.8.10!

By using this community edition, you agree with the license available at https://docs.atoti.io/latest/eula.html.
Browse the official documentation at https://docs.atoti.io.
Join the community at https://www.atoti.io/register.

Atoti collects telemetry data, which is used to help understand how to improve the product.
If you don't wish to send usage data, you can request a trial license at https://www.atoti.io/evaluation-license-request.

You can hide this message by setting the `ATOTI_HIDE_EULA_MESSAGE` environment variable to True.


In [3]:


# Read the tables into the atoti session
athlete_table = session.read_sql(
    "SELECT * FROM dimathlete",
    keys=["athlete_id"],
    table_name="Athlete",
    url=jdbc_url,
)

athlete_country_table = session.read_sql(
    "SELECT * FROM dimathlete_country",
    keys=["athlete_countrycode"],
    table_name="AthleteCountry",
    url=jdbc_url,
)

event_table = session.read_sql(
    "SELECT * FROM dimevent",
    keys=["event_id"],
    table_name="Event",
    url=jdbc_url,
)

game_table = session.read_sql(
    "SELECT * FROM dimgame",
    keys=["game_id"],
    table_name="Game",
    url=jdbc_url,
)

year_table = session.read_sql(
    "SELECT * FROM dimyear",
    keys=["year_id"],
    table_name="Year",
    types = {'year' : tt.type.STRING},
    url=jdbc_url,
)

economy_table = session.read_sql(
    "SELECT * FROM dimeconomy",
    keys=["economy_id"],
    table_name="Economy",
    url=jdbc_url,
)

medal_table = session.read_sql(
    "SELECT * FROM dimmedal",
    keys=["medal_id"],
    table_name="Medal",
    url=jdbc_url,
)

fact_olympic_athletes_table = session.read_sql(
    "SELECT * FROM fact_olympicathletes",
    keys=["record_id"],
    table_name="OlympicAthletes",
    url=jdbc_url,
)


### Implement Star Schema

In [4]:
fact_olympic_athletes_table.join(athlete_table, fact_olympic_athletes_table["athlete_id"] == athlete_table["athlete_id"])
fact_olympic_athletes_table.join(athlete_country_table, fact_olympic_athletes_table["athlete_countrycode"] == athlete_country_table["athlete_countrycode"])
fact_olympic_athletes_table.join(event_table, fact_olympic_athletes_table["event_id"] == event_table["event_id"])
fact_olympic_athletes_table.join(game_table, fact_olympic_athletes_table["game_id"] == game_table["game_id"])
fact_olympic_athletes_table.join(year_table, fact_olympic_athletes_table["year_id"] == year_table["year_id"])
fact_olympic_athletes_table.join(economy_table, fact_olympic_athletes_table["economy_id"] == economy_table["economy_id"])
fact_olympic_athletes_table.join(medal_table, fact_olympic_athletes_table["medal_id"] == medal_table["medal_id"])

In [5]:
fact_olympic_athletes_table.head()

,athlete_id,athlete_countrycode,event_id,medal_id,game_id,year_id,economy_id,gold_medals,silver_medals,bronze_medals
record_id,,,,,,,,,,
51,50,USA ...,15,1,beijing-2022,1,USA008,1,0,0
53,52,CAN ...,16,3,beijing-2022,1,CAN006,0,0,1
61,60,CHN ...,19,1,beijing-2022,1,CHN011,1,0,0
10,9,SWE ...,2,1,beijing-2022,1,SWE003,1,0,0
69,55,HUN ...,21,3,beijing-2022,1,HUN019,0,0,1


In [6]:
session.tables.schema

```mermaid
erDiagram
  "Game" {
    _ String PK "game_id"
    _ String "game_name"
    _ String "gamelocation"
  }
  "Economy" {
    _ String PK "economy_id"
    nullable double "health_expenditure"
    nullable double "gdp_percapita"
  }
  "Year" {
    _ int PK "year_id"
    _ String "year"
    _ String "season"
  }
  "OlympicAthletes" {
    _ int PK "record_id"
    nullable int "athlete_id"
    _ String "athlete_countrycode"
    nullable int "event_id"
    nullable int "medal_id"
    _ String "game_id"
    nullable int "year_id"
    _ String "economy_id"
    nullable int "gold_medals"
    nullable int "silver_medals"
    nullable int "bronze_medals"
  }
  "Event" {
    _ int PK "event_id"
    _ String "title"
    _ String "event_gender"
    _ String "discipline"
  }
  "Medal" {
    _ int PK "medal_id"
    _ String "medal_type"
  }
  "Athlete" {
    _ int PK "athlete_id"
    _ String "athlete_name"
    _ String "gender"
  }
  "AthleteCountry" {
    _ String PK "athlete_countrycode"
    _ String "country_name"
    _ String "continent"
  }
  "OlympicAthletes" }o--o| "Economy" : "`economy_id` == `economy_id`"
  "OlympicAthletes" }o--o| "Event" : "`event_id` == `event_id`"
  "OlympicAthletes" }o--o| "AthleteCountry" : "`athlete_countrycode` == `athlete_countrycode`"
  "OlympicAthletes" }o--o| "Athlete" : "`athlete_id` == `athlete_id`"
  "OlympicAthletes" }o--o| "Medal" : "`medal_id` == `medal_id`"
  "OlympicAthletes" }o--o| "Year" : "`year_id` == `year_id`"
  "OlympicAthletes" }o--o| "Game" : "`game_id` == `game_id`"
```


### Cube

In [7]:
cube = session.create_cube(fact_olympic_athletes_table)
cube

In [8]:
hierarchies, levels, measures = cube.hierarchies, cube.levels, cube.measures
hierarchies

{('OlympicAthletes', 'record_id'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB7E5850>, ('AthleteCountry', 'country_name'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB7E58B0>, ('Game', 'gamelocation'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB734BE0>, ('Event', 'title'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB7D7910>, ('Event', 'event_gender'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB4F5460>, ('Year', 'year'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB7F4070>, ('Event', 'discipline'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB7523A0>, ('OlympicAthletes', 'game_id'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB715400>, ('Athlete', 'athlete_name'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB7E5FA0>, ('OlympicAthletes', 'athlete_countrycode'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB6FA580>, ('Athlete', 'gender'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB6FA670>, ('AthleteCountry', 'continent'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB6FA8B0>, ('OlympicAthletes', 'economy_id'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB7405E0>, ('Game', 'game_name'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB740C10>, ('Year', 'season'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB740AC0>, ('Medal', 'medal_type'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB7408B0>}

In [9]:
levels

In [10]:
measures

{'contributors.COUNT': <atoti.measure.Measure object at 0x0000010FBB7A90D0>, 'update.TIMESTAMP': <atoti.measure.Measure object at 0x0000010FBB94A7C0>, 'gold_medals.MEAN': <atoti.measure.Measure object at 0x0000010FBB94AAF0>, 'gold_medals.SUM': <atoti.measure.Measure object at 0x0000010FBB94A880>, 'year_id.MEAN': <atoti.measure.Measure object at 0x0000010FBB94A9A0>, 'year_id.SUM': <atoti.measure.Measure object at 0x0000010FBB94A820>, 'athlete_id.MEAN': <atoti.measure.Measure object at 0x0000010FBB94A4C0>, 'athlete_id.SUM': <atoti.measure.Measure object at 0x0000010FBB94A460>, 'bronze_medals.MEAN': <atoti.measure.Measure object at 0x0000010FBB94A070>, 'bronze_medals.SUM': <atoti.measure.Measure object at 0x0000010FBB94A550>, 'silver_medals.MEAN': <atoti.measure.Measure object at 0x0000010FBB94A100>, 'silver_medals.SUM': <atoti.measure.Measure object at 0x0000010FBB9C85E0>, 'event_id.MEAN': <atoti.measure.Measure object at 0x0000010FBB9C86D0>, 'event_id.SUM': <atoti.measure.Measure object at 0x0000010FBB9C8730>, 'medal_id.MEAN': <atoti.measure.Measure object at 0x0000010FBB9C8790>, 'medal_id.SUM': <atoti.measure.Measure object at 0x0000010FBB9C8700>}

In [11]:
economy_table.head()

,health_expenditure,gdp_percapita
economy_id,,
SWE003,11.379928,52837.90398
CHN011,5.593597,10408.71912
HUN019,7.250314,16125.60941
LVA027,7.44663,18207.13964
KWT035,6.307433,24297.70114


### Clean Hierachies, Levels and Measures

In [12]:
hierarchies["Athlete"] = [levels["athlete_name"], levels["gender"]]
hierarchies["AthleteCountry"] = [levels["country_name"], levels["continent"]]
hierarchies["Event"] = [levels["event_gender"], levels["title"], levels["discipline"]]
hierarchies["Game"] = [levels["game_name"], levels["gamelocation"]]
hierarchies["Medal"] = [levels["medal_type"]]
hierarchies["Year"] = [levels["season"], levels["year"]] 


In [13]:
hierarchies

{('OlympicAthletes', 'record_id'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB740A00>, ('AthleteCountry', 'country_name'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB7402E0>, ('Game', 'gamelocation'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB740B20>, ('Event', 'title'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB7409A0>, ('Event', 'event_gender'): <atoti.hierarchy.Hierarchy object at 0x0000010FBAFEB730>, ('Year', 'year'): <atoti.hierarchy.Hierarchy object at 0x0000010FAA6449A0>, ('Event', 'discipline'): <atoti.hierarchy.Hierarchy object at 0x0000010FBAFEB820>, ('OlympicAthletes', 'game_id'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB82BC70>, ('Athlete', 'athlete_name'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB82B730>, ('OlympicAthletes', 'athlete_countrycode'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB82B700>, ('Athlete', 'gender'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB4E3EB0>, ('AthleteCountry', 'continent'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB4E3550>, ('OlympicAthletes', 'economy_id'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB4E3850>, ('Game', 'Game'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB740E80>, ('Year', 'Year'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB740730>, ('AthleteCountry', 'AthleteCountry'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB6FA580>, ('Game', 'game_name'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB6FA4C0>, ('Year', 'season'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB95F7C0>, ('Medal', 'medal_type'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB95FFA0>, ('Medal', 'Medal'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB7BA4F0>, ('Event', 'Event'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB8340D0>, ('Athlete', 'Athlete'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB950280>}

In [14]:
# clean Athlete Hierachies:
del hierarchies[('Athlete', 'athlete_name')]
del hierarchies[('Athlete', 'gender')]
# clean Athlete Country Hierachies:
del hierarchies[('AthleteCountry', 'continent')]
del hierarchies[('AthleteCountry', 'country_name')]
# clean Event Hierachies:
del hierarchies[('Event', 'discipline')]
del hierarchies[('Event', 'event_gender')]
del hierarchies[('Event', 'title')]

# clean Game Hierarchies:
del hierarchies[('Game', 'game_name')]
del hierarchies[('Game', 'gamelocation')]

# clean Medal Hierachies:
del hierarchies[('Medal', 'medal_type')]

# clean Year Hierachies:
del hierarchies[('Year', 'season')]
del hierarchies[('Year', 'year')]


# remove fact table ids:
del hierarchies[('OlympicAthletes', 'athlete_countrycode')]
del hierarchies[('OlympicAthletes', 'economy_id')]
del hierarchies[('OlympicAthletes', 'game_id')]
del hierarchies[('OlympicAthletes', 'record_id')]

In [15]:
measures

{'contributors.COUNT': <atoti.measure.Measure object at 0x0000010FBB9D1460>, 'update.TIMESTAMP': <atoti.measure.Measure object at 0x0000010FBB8140D0>, 'gold_medals.MEAN': <atoti.measure.Measure object at 0x0000010FBB814DC0>, 'gold_medals.SUM': <atoti.measure.Measure object at 0x0000010FBB8148E0>, 'year_id.MEAN': <atoti.measure.Measure object at 0x0000010FBB8144C0>, 'year_id.SUM': <atoti.measure.Measure object at 0x0000010FBB814160>, 'athlete_id.MEAN': <atoti.measure.Measure object at 0x0000010FBB7BFBE0>, 'athlete_id.SUM': <atoti.measure.Measure object at 0x0000010FBB7BFD30>, 'bronze_medals.MEAN': <atoti.measure.Measure object at 0x0000010FBB7BFB80>, 'bronze_medals.SUM': <atoti.measure.Measure object at 0x0000010FBB7BF6D0>, 'silver_medals.MEAN': <atoti.measure.Measure object at 0x0000010FBB7BF1F0>, 'silver_medals.SUM': <atoti.measure.Measure object at 0x0000010FBB7BFA60>, 'event_id.MEAN': <atoti.measure.Measure object at 0x0000010FBB7BF700>, 'event_id.SUM': <atoti.measure.Measure object at 0x0000010FBB7BFAC0>, 'medal_id.MEAN': <atoti.measure.Measure object at 0x0000010FBB7BFC70>, 'medal_id.SUM': <atoti.measure.Measure object at 0x0000010FBB7BF280>}

In [16]:
# clean measures:
del measures["athlete_id.MEAN"]
del measures["athlete_id.SUM"]
del measures["bronze_medals.MEAN"]
del measures["contributors.COUNT"]
del measures["event_id.MEAN"]
del measures["event_id.SUM"]
del measures["gold_medals.MEAN"]
#del measures["gold_medals.SUM"]
del measures["medal_id.MEAN"]
del measures["medal_id.SUM"]
del measures["silver_medals.MEAN"]
del measures["year_id.MEAN"]
del measures["year_id.SUM"]

In [17]:
cube

In [18]:
hierarchies["Year"] = [levels[('Year', 'year')], levels[('Year', 'season')]]
hierarchies

{('Game', 'Game'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB7BFBB0>, ('Year', 'Year'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB7BF340>, ('AthleteCountry', 'AthleteCountry'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB7BF4F0>, ('Medal', 'Medal'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB7BF0A0>, ('Event', 'Event'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB7D7580>, ('Athlete', 'Athlete'): <atoti.hierarchy.Hierarchy object at 0x0000010FBB7D7220>}

In [19]:
levels

### Create Measures:

In [20]:
# create measures GDP and health expenditure 
measures["GDP per Capita"] = tt.agg.sum(economy_table["gdp_percapita"])
measures["Health Expenditure"] = tt.agg.sum(economy_table["health_expenditure"])
# Create a new measure for the total medals
measures["Total Medals.SUM"] = measures["gold_medals.SUM"] + measures["silver_medals.SUM"] + measures["bronze_medals.SUM"]



### Query Measures

In [29]:
cube.query(measures["gold_medals.SUM"], levels=[levels["discipline"]])

gold_medals.SUM
event_gender                                       title                          discipline                         
Men                                            ... + 100kg (heavyweight) men      Judo                              2
                                                   + 80 kg men                    Taekwondo                         1
                                                   + 91kg (super heavyweight) men Boxing                            2
                                                   + 95kg (heavyweight) men       Judo                              1
                                                   +105kg men                     Weightlifting                     1
...                                                                                                               ...
Women                                          ... vault women                    Gymnastics Artistic              18
                                                   volleyball women               Volleyball                       14
                                                   water polo women               Water Polo                        5
                                                   épée individual women          Fencing                           6
                                                   épée team women                Fencing                           5

[1588 rows x 1 columns]

In [22]:
cube.query(measures["Total Medals.SUM"], levels=[levels["continent"]])

,,Total Medals.SUM
country_name,continent,
Afghanistan,Asia,2
Algeria,Africa,17
Argentina,South America,87
Armenia,Asia,18
Australasia,Oceania,12
...,...,...
Vietnam,Asia,4
Virgin Islands (U.S.),North America,1
West Indies Federation,North America/Caribbean,2


In [38]:
cube.query(measures["GDP per Capita"], levels=[levels["continent"]])

,,GDP per Capita
country_name,continent,
Afghanistan,Asia,"1,033.73"
Algeria,Africa,"57,020.67"
Argentina,South America,"739,189.25"
Armenia,Asia,"81,105.62"
Australasia,Oceania,.00
...,...,...
Vietnam,Asia,"14,345.39"
Virgin Islands (U.S.),North America,"39,552.17"
West Indies Federation,North America/Caribbean,.00


In [24]:
session.tables.schema

```mermaid
erDiagram
  "Game" {
    _ String PK "game_id"
    _ String "game_name"
    _ String "gamelocation"
  }
  "Economy" {
    _ String PK "economy_id"
    nullable double "health_expenditure"
    nullable double "gdp_percapita"
  }
  "Year" {
    _ int PK "year_id"
    _ String "year"
    _ String "season"
  }
  "OlympicAthletes" {
    _ int PK "record_id"
    nullable int "athlete_id"
    _ String "athlete_countrycode"
    nullable int "event_id"
    nullable int "medal_id"
    _ String "game_id"
    nullable int "year_id"
    _ String "economy_id"
    nullable int "gold_medals"
    nullable int "silver_medals"
    nullable int "bronze_medals"
  }
  "Event" {
    _ int PK "event_id"
    _ String "title"
    _ String "event_gender"
    _ String "discipline"
  }
  "Medal" {
    _ int PK "medal_id"
    _ String "medal_type"
  }
  "Athlete" {
    _ int PK "athlete_id"
    _ String "athlete_name"
    _ String "gender"
  }
  "AthleteCountry" {
    _ String PK "athlete_countrycode"
    _ String "country_name"
    _ String "continent"
  }
  "OlympicAthletes" }o--o| "Economy" : "`economy_id` == `economy_id`"
  "OlympicAthletes" }o--o| "Event" : "`event_id` == `event_id`"
  "OlympicAthletes" }o--o| "AthleteCountry" : "`athlete_countrycode` == `athlete_countrycode`"
  "OlympicAthletes" }o--o| "Athlete" : "`athlete_id` == `athlete_id`"
  "OlympicAthletes" }o--o| "Medal" : "`medal_id` == `medal_id`"
  "OlympicAthletes" }o--o| "Year" : "`year_id` == `year_id`"
  "OlympicAthletes" }o--o| "Game" : "`game_id` == `game_id`"
```
